### Importieren der notwendigen Klassen

In [1]:
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter import scrolledtext
from tkinter import filedialog
from random import choice

### Konstanten definieren

In [2]:
NochImSpiel = 0
KI_gewonnen = 1
KI_verloren = 2

name = {True:"Bob, die KI", False:"Alice"}

### Die zentrale Klasse

In [3]:
class muenzen():
    
    def __init__(self, dim = 5, take = 2):
        self.dim = dim # wieviele Münzen liegen beim Start auf dem Tisch?
        self.take = take # wieviele Münzen darf man (maximal) nehmen?
        self.muenzenAufTisch = dim
        self.sinnvolleZuege = {}
        self.letzterGemerkterIndex = 0
        self.memory = {}
        for i in range(1, self. dim + 1):
            self.sinnvolleZuege[i] = [x for x in range(1,take+1) if x <= i]
            self.memory[i] = None # nichts gemerkt
        self.ende_status = NochImSpiel # das Spiel läuft noch
        self.begin = False # False = der Gegner; True = die KI
        self.am_zug = self.begin
        self.comment = ""

    # Ein neuer Durchgang, falls einer der beiden verloren hat
    # Der jeweils andere Spieler beginnt dann
    def neueRunde(self):
        if self.ende_status == NochImSpiel:
            self.comment = "Spiel noch nicht beendet"
            return

        self.muenzenAufTisch = self.dim
        if self.ende_status == KI_gewonnen:
            self.KI_memory_back()
        elif self.ende_status == KI_verloren:
            for i in range(1, self. dim + 1):
                self.memory[i] = None
            self.letzterGemerkterIndex = 0
            
        self.ende_status = NochImSpiel
        self.begin = not self.begin # jetzt fängt der jeweils andere an
        self.am_zug = self.begin
        self.comment = "Reset erfolgt!"
        
    def KI_merken(self, muenzen, zug):
        self.comment = ""
        if zug in self.sinnvolleZuege[muenzen]:
            self.sinnvolleZuege[muenzen].remove(zug)
            self.memory[muenzen] = zug
            self.letzterGemerkterIndex = muenzen
            self.comment = f"gemerkt von Zug {zug} bei {muenzen} Münzen!"

        else:
            self.comment = f"KI_merken von Zug {zug} bei {muenzen} Münzen nicht erlaubt!"
            
    def KI_memory_back(self):
        if self.letzterGemerkterIndex != 0:
            mem = self.memory[self.letzterGemerkterIndex]
            if mem != None:
                self.memory[self.letzterGemerkterIndex] = None
                self.sinnvolleZuege[self.letzterGemerkterIndex].append(mem)
                self.letzterGemerkterIndex = 0

    def sinnvolle_KI_zuege(self, anzahl_muenzen):
        zuege = []
        for zug in self.sinnvolleZuege[anzahl_muenzen]:
            zuege.append(zug)
        return zuege
    
    def moegliche_KI_zuege(self, anzahl_muenzen):
        if self.sinnvolleZuege[anzahl_muenzen] == []:
            return [x for x in range(1,self.take+1) if x <= anzahl_muenzen]
        else:
            return self.sinnvolle_KI_zuege(anzahl_muenzen)
    
    def moegliche_human_zuege(self, anzahl_muenzen):
        return [x for x in range(1,self.take+1) if x <= anzahl_muenzen]
        
    def zufalls_Human_Zug(self):
        anzahl_muenzen = self.muenzenAufTisch
        return choice(self.moegliche_human_zuege(anzahl_muenzen))
    
    def autoZugMachenHuman(self):
        anz = self.zufalls_Human_Zug()
        self.wegnehmen(anz)
    
    def zufalls_KI_Zug(self):
        anzahl_muenzen = self.muenzenAufTisch
        return choice(self.moegliche_KI_zuege(anzahl_muenzen))
    
    def autoZugMachenKI(self):
        anz = self.zufalls_KI_Zug()
        self.KI_move(anz)
        
    def autoZugMachen(self):
        if self.am_zug:
            self.autoZugMachenKI()
        else:
            self.autoZugMachenHuman()
        

################################                
    # Das ist nur für Alice!!!
    # Und geht nur, wenn der Zug möglich ist!
    # Nachdem die Münzen genommen wurden, wechselt der Spieler!
    def wegnehmen(self, anzahl): # Alice nimmt
        self.comment = ""
        if self.am_zug:
            self.comment = f"{name[False]} ist nicht am Zug!"
            return
        
        if anzahl > self.muenzenAufTisch:
            self.comment = f"Wegnehmen von {anzahl} Münzen nicht erlaubt!"
            return
            
        self.comment = f"{name[False]} hat {anzahl} Münzen genommen."
        self.muenzenAufTisch -= anzahl
            
        
        if self.muenzenAufTisch == 0: # Alice hat gewonnen
            self.ende_status = KI_verloren
            if self.letzterGemerkterIndex != 0:
                self.memory[self.letzterGemerkterIndex] = None
        else:
            self.am_zug = True # jetzt ist KI dran

################################
    # Das geht nur für die KI!
    # Und nur, wenn der Zug möglich ist!
    # Falls noch Zug im Vorrat, dann 
    #   gemerkter Zug zurück
    #   zug merken
    #   Zug machen
    # Sonst
    #   gemerkter Zug weg!
    #   Zug machen
    # Spielende beachten
    # Spieler wechselt
    
    def KI_move(self, anzahl):
        if not self.am_zug:
            self.comment = "KI ist nicht am Zug!"
            return
        
        if anzahl > self.muenzenAufTisch:
            self.comment = f"KI kann nicht {anzahl} Münzen nehmen!"
            return
               
        if self.sinnvolleZuege[self.muenzenAufTisch] == []: 
            self.memory[self.letzterGemerkterIndex] = None
            self.letzterGemerkterIndex = 0
            self.muenzenAufTisch -= anzahl
            self.comment = f"Kein Sinnvoller Zug mehr möglich. {name[True]} hat {anzahl} Münzen genommen."

            if self.muenzenAufTisch == 0: # KI hat gewonnen; das ist jedoch eigentlich nicht möglich,
                                          # da ansonsten dieser Zug noch machbar gewesen wäre!
                self.ende_status = KI_gewonnen
                self.comment += "/nSeltsam!!!"
            else:
                self.am_zug = False # jetzt ist Alice dran
        else:   
            if anzahl in self.sinnvolle_KI_zuege(self.muenzenAufTisch):
                if self.letzterGemerkterIndex != 0:
                    self.KI_memory_back()
                    
                self.KI_merken(self.muenzenAufTisch, anzahl)
                self.muenzenAufTisch -= anzahl
                self.comment = f"{name[True]} hat {anzahl} Münzen genommen. Ist ggf. sinnvoll?."
                  
                if self.muenzenAufTisch == 0: # Ich habe gewonnen
                    self.ende_status = KI_gewonnen
                else:
                    self.am_zug = False # jetzt ist Alice dran
            else:
                self.comment = f"Zug {anzahl} nicht erlaubt!"            
            
    def spiel_status_text(self):
        if self.ende_status == NochImSpiel:
            return f"Am Zug ist {name[self.am_zug]}"
        elif self.ende_status == KI_gewonnen:
            return name[True] + " hat gewonnen!"
        else:
            return name[True] + " hat verloren!"
        
    def out_board(self):
        out = ""
        if self.muenzenAufTisch == 0:
            out += (f"Es ist keine Münze mehr auf dem Tisch\n\n")
        elif self.muenzenAufTisch == 1:
            out +=(f"Es ist {self.muenzenAufTisch} Münze auf dem Tisch\n\n")
        else:
            out +=(f"Es sind {self.muenzenAufTisch} Münzen auf dem Tisch\n\n")
        
        for i in range(self.dim,0,-1):
            if self.memory[i] == None:
                past = ""
            else:
                past = "Genommen " + str(self.memory[i])
            if i == self.muenzenAufTisch:
                pre = "➔  "
            else:
                pre = "     "
                
            zugliste = self.sinnvolleZuege[i]
            zl = list (map (lambda x : "Nimm " + str(x), zugliste))
            zl_string = "[" + ", ".join(zl) + "]"
            out +=(f"{pre}Status: {i} Münze(n)\t\t sinnvolle Züge: {zl_string}\t\t {past}\n")
        
        out += m.spiel_status_text()

        if self.muenzenAufTisch != 0:
            mz = "machbare"
            if self.am_zug:
                zuege = self.moegliche_KI_zuege(self.muenzenAufTisch)
                if self.sinnvolleZuege[self.muenzenAufTisch] != []:
                    mz = "sinnvolle"
            else:
                zuege = self.moegliche_human_zuege(self.muenzenAufTisch)
                            
            out +=(f"\nFür {name[self.am_zug]} {mz} Züge: {zuege}\n")
        
        return out
            
    def print_board(self):
        print("Comment: " + self.comment)
        print(self.out_board())
        print("--------")
        
    def eineAutoRunde(self):
        while m.ende_status == NochImSpiel:
            m.autoZugMachen()

    def training(self, anzahlRunden):
        self.neueRunde()
        for _ in range(anzahlRunden):
            self.eineAutoRunde()
            self.neueRunde()

## Und hier interaktiv

### Die GUI

In [4]:
dim = 5
take = 2

m = muenzen(dim, take)


def newAction():
    global dim
    global take
    global m

    dim = int(dimSelectedNumber.get())
    take = int(takeSelectedNumber.get())

    m = muenzen(dim, take)

    if trainVar.get() == 1:
        m.training(anzTrain.get())
    out_state = m.out_board()
    text_area.config(state="normal")
    text_area.delete('0.0', END)
    text_area.insert(INSERT, out_state + "\n")
    text_area.config(state="disabled")

    #   out = m.comment
    #   comment_area.config(state = "normal")
    #   comment_area.delete('0.0', END)
    #   comment_area.insert(INSERT,out)
    #   comment_area.config(state = "disabled")

    labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    labelZug.config(text=labText)

    if m.ende_status == NochImSpiel:  # das sollte doch so sein!!!
        if m.am_zug:
            allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
        else:
            allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
        zug_eingabebox['values'] = allowed
        zug_eingabebox.set('')
    else:
        zug_eingabebox.set('')


def newRoundAction():
    m.neueRunde()
    reset_button.config(state="disabled")
    zug_eingabebox.config(state='readonly')

    out_state = m.out_board()
    text_area.config(state="normal")
    text_area.delete('0.0', END)
    text_area.insert(INSERT, out_state + "\n")
    text_area.config(state="disabled")

    #   out = m.comment
    #   comment_area.config(state = "normal")
    #   comment_area.delete('0.0', END)
    #   comment_area.insert(INSERT,out)
    #   comment_area.config(state = "disabled")

    labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    labelZug.config(text=labText)

    if m.ende_status == NochImSpiel:  # das sollte doch so sein!!!
        if m.am_zug:
            allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
        else:
            allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
        zug_eingabebox['values'] = allowed
        zug_eingabebox.set('')
    else:
        zug_eingabebox.set('')


def act(event):
    zahltext = selected_Zug.get()

    if not m.am_zug:
        m.wegnehmen(int(zahltext))
    else:
        m.KI_move(int(zahltext))

    out_state = m.out_board()
    text_area.config(state="normal")
    text_area.delete('0.0', END)
    text_area.insert(INSERT, out_state)
    text_area.config(state="disabled")

    #   out = m.comment
    #   if out == "":
    #       out = "No Comment\n"
    #   comment_area.config(state = "normal")
    #   comment_area.insert(INSERT,out + '\n')
    #   comment_area.config(state = "disabled")

    labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"
    labelZug.config(text=labText)

    if m.ende_status == NochImSpiel:
        if m.am_zug:
            allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
        else:
            allowed = m.moegliche_human_zuege(m.muenzenAufTisch)
        zug_eingabebox['values'] = allowed
        zug_eingabebox.set('')
    else:
        reset_button.config(state="normal")
        zug_eingabebox.set('')
        zug_eingabebox.config(state="disabled")


################## MainFrame ###############################

mainFrame = Tk()
mainFrame.title("Münzspiel: Alice gegen Bob, die KI")
mainFrame.resizable()

################ Kommentar #################################

commLabel = Label(mainFrame, width=40)
commLabText = f"Der Spieler, der die letzte Münze nimmt, gewinnt!"
commLabel['anchor'] = "c"

commLabel.config(
    text=commLabText,
    borderwidth=2,
    wraplength=300,
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    relief='raised',
    border='2',
)

commLabel.pack(padx=5, pady=5, side=TOP)

################ NewFrame #################################

newFrame = Frame(mainFrame, background="gray")
newFrame.pack(padx=5, pady=5, side=TOP)

################## dimFrame ###############################

confDimFrame = Frame(newFrame)
confDimFrame.pack(padx=5, pady=5, side=TOP)

dimLabel = Label(confDimFrame, width=40)
dimLabText = f"Wieviele Münzen liegen beim Start auf dem Tisch"
dimLabel['anchor'] = "e"

dimLabel.config(
    text=dimLabText, 
    borderwidth=2,
    wraplength=300,
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    relief='raised',
    border='2',
    )

dimLabel.pack(padx=5, pady=5, side=LEFT)

dimSelectedNumber = StringVar()
dimNumberEingabebox = ttk.Combobox(
    confDimFrame,
    textvariable=dimSelectedNumber,
)

dimNumberEingabebox['values'] = list(range(2, 12))
dimNumberEingabebox.set(str(dim))
dimNumberEingabebox.pack(padx=5, pady=5, side=RIGHT)

dimNumberEingabebox['state'] = 'readonly'

#################### TakeFrame #############################

confTakeFrame = Frame(newFrame)
confTakeFrame.pack(padx=5, pady=5, side=TOP)

takeLabel = Label(confTakeFrame, width=40)
takeLabText = f"Wieviele Münzen darf man maximal nehmen?"
takeLabel['anchor'] = "e"

takeLabel.config(text=takeLabText, 
    borderwidth=2,
    wraplength=300,
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    relief='raised',
    border='2',
    )
                

takeLabel.pack(padx=5, pady=5, side=LEFT)

takeSelectedNumber = StringVar()
takeNumberEingabebox = ttk.Combobox(
    confTakeFrame,
    textvariable=takeSelectedNumber,
)

takeNumberEingabebox['values'] = list(range(2, 5))
takeNumberEingabebox.set(str(take))
takeNumberEingabebox.pack(padx=5, pady=5, side=RIGHT)

takeNumberEingabebox['state'] = 'readonly'

################ NewButtonFrame #################################

newButtonFrame = Frame(newFrame)
newButtonFrame.pack(padx=5, pady=5, side=TOP)

newButton = Button(
    newButtonFrame,
    text="ganz neu mit diesen Werten",
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    command=newAction,
    state='activ',
                          #relief = "raised", # geht in MACOS nicht!!!
)

newButton.pack(padx=5, pady=5, side=LEFT)

trainVar = IntVar()
mitTrain = Checkbutton(
    newButtonFrame,
    text='mit Training',
    font =('Helvetica', 14, 'bold'),
    variable=trainVar,
    onvalue=1,
    offvalue=0,
)

mitTrain.pack(padx=5, pady=5, side=LEFT)

anzTrain = Scale(newButtonFrame,
                 from_=0,
                 to=300,
                 orient=VERTICAL,
                 label="Anzahl Trainingsrunden")

anzTrain.set(10)
anzTrain.pack(padx=5, pady=5, side=LEFT)

##################### Output Frame ############################

text_ausgabe = Frame(mainFrame)
text_ausgabe.pack(padx=5, pady=5, side=TOP)

gehirnLabel = Label(text_ausgabe, width=2)
gehirnText = f"Gedächtnis"
#gehirnLabel['anchor']="e"

gehirnLabel.config(
    text=gehirnText,
    borderwidth=2,
    wraplength=1,
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    relief='raised',
    border='2',
)

gehirnLabel.pack(padx=5, pady=5, side=LEFT)

slider_textfield = Scrollbar(text_ausgabe, orient=VERTICAL)

text_area = Text(text_ausgabe,
                 wrap=WORD,
                 width=80,
                 height=15,
                 font=("Times New Roman", 18, 'bold'),
                 state='normal',
                 yscrollcommand=slider_textfield.set,
                 relief='sunken',
                 borderwidth=2)

out_state = m.out_board()
text_area.insert(INSERT, out_state)
text_area.config(state="disabled")
slider_textfield.config(command=text_area.yview)

slider_textfield.pack(padx=5, pady=5, side=RIGHT, fill=Y)

text_area.pack(pady=5, padx=5)

##################### InputFrame ############################

inputFrame = Frame(mainFrame, background="gray")
inputFrame.pack(padx=5, pady=5, side=TOP)

##################### Eingabe ############################

eingabe = Frame(inputFrame)
eingabe.pack(padx=5, pady=5, side=TOP)

labelZug = Label(eingabe, width=40)
labelZug['anchor'] = "e"

labText = f"Wieviele Münzen nimmt {name[m.am_zug]}?"

labelZug.config(text=labText, 
    borderwidth=2,
    wraplength=300,
    font=('Helvetica', 14, 'bold'),
#    fg='red',
    anchor="center",    
    relief='raised',
    border='2',
    )
               

labelZug.pack(padx=5, pady=5, side=LEFT)

selected_Zug = StringVar()
zug_eingabebox = ttk.Combobox(
    eingabe,
    textvariable=selected_Zug,
)

zug_eingabebox['state'] = 'readonly'

if m.am_zug:
    allowed = m.moegliche_KI_zuege(m.muenzenAufTisch)
else:
    allowed = m.moegliche_human_zuege(m.muenzenAufTisch)

zug_eingabebox['values'] = allowed
zug_eingabebox.set('')
zug_eingabebox.bind('<<ComboboxSelected>>', act)
zug_eingabebox.pack(padx=5, pady=5, side=RIGHT)

##################### neue Runde Frame ############################

newRoundFrame = Frame(mainFrame, background="white")
newRoundFrame.pack(padx=5, pady=5, side=TOP)

reset_button = Button(
    mainFrame,
    text="neue Runde",
    command=newRoundAction,
    font=('Helvetica', 14, 'bold'),
    state='disabled',
    relief="raised",  # geht in MACOS nicht!!!
)

reset_button.pack(padx=5, pady=5, side=TOP)

##################### Comment Frame ############################

comment_ausgabe = Frame(mainFrame)
comment_ausgabe.pack(padx=5, pady=5)

slider_commentfield = Scrollbar(comment_ausgabe, orient=VERTICAL)

comment_area = Text(
    comment_ausgabe,
    wrap=WORD,
    width=40,
    height=10,
    font=("Helvetica", 14),
    state='normal',
    yscrollcommand=slider_commentfield.set,
    relief='sunken',
    borderwidth=2,
)

comment_area.insert(INSERT, "NoComment\n")
comment_area.config(state="disabled")
# slider_commentfield.config(command = comment_area.yview)

# slider_commentfield.pack(side = RIGHT, fill = Y)

#comment_area.pack(padx = 5, pady = 5)

##################### Main ############################

mainFrame.mainloop()